
<div style="text-align: center; padding: 30px 10px;">

<h2 style="font-size: 30px; margin-top: 5px;">
Лабораторная работа 1. О смысле жизни.
</h2>

<hr style="width: 60%; border: 1px solid #10069f; margin: 25px auto;">

</div>

Если Вы видите <b><font color="#FF69B4"> Ваш ответ здесь </font></b> - надо поменять текст внутри блока на свой текстовый ответ на вопрос.


ВАЖНО! Здесь и далее от вас ожидается чистый код, близкий к тому, что нужно в индустрии. В контексте данной лабораторной работы ожидаются корректные названия у полей, методов и переменных, которые вы вводите; корректная инкапсуляция - нижнее подчеркивание или два перед названием метода или атрибута. Для корректной инкапсуляции можно (и даже нужно) добавлять нижние подчеркивания даже к названиям методов/атрибутов из условий задач :)

# 1. Абстракции (3 балла)

Напиши абстрактный класс `Organism`. От него будут наследоваться другие классы.

В нём должен быть:

Concrete Methods:

- `property` - `alive`, который проверяет, что текущий возраст не больше, чем максимальный и энергия не отрицательная. При попытке изменения (сеттер), нужно: если статус "живости" не изменяется, то ничего не меняем; если мёртвый -> живой, то возраст - максимальный - 1, энергия - 1; если живой -> мёртвый, то возраст делаем максимальным + 1.
- Мэджик метод, позволяющий использовать экземпляры в if-конструкциях (мэджик метод `__bool__`, связанный с `alive`).
- Инициализатор (`__init__`), который проставляет атрибуты экземпляра: размер популяции `population_size = 0`, текущий возраст `current_age = 0`, `current_energy = 1` и `id` в зависимости от текущей глобальной переменной `ID_COUNTER`. Последнюю надо увеличить на 1, чтобы последующие ID отличались.
- Метод класса (`classmethod`) `spawn`, принимающий на вход `population_size`, `current_age` и фиксирующий их для объекта.
- Метод `reproduce`, который возвращает новый объект данного класса, в котором `current_age = 0`, `current_energy` - меньше в `REPRODUCE_NEW_COEF` раз, округлённых вниз; population_size - меньше в `REPRODUCE_POPULATION_COEF` раз, округлённых вниз. Кроме того, энергия изначального объекта понижается в `REPRODUCE_OLD_COEF` раз и логгирует в поток вывода сообщение с ID тех, кто размножился и тех, кто появился.


Abstract Methods:

- Мэджик метод, отвечающий за репрезентацию объекта (str/repr)
- Метод `replenish_energy`, отвечающий за пополнение энергии



In [2]:
from abc import ABC, abstractmethod
import math

ID_COUNTER = 0
REPRODUCE_NEW_COEF = 2
REPRODUCE_POPULATION_COEF = 2
REPRODUCE_OLD_COEF = 2

class Organism(ABC):
    def __init__(self):
        global ID_COUNTER
        self._population_size = 0
        self._current_age = 0
        self._current_energy = 1
        self._id = ID_COUNTER
        ID_COUNTER += 1
        self._max_age = None

    @property
    def alive(self):
        return self._current_age <= self._max_age and self._current_energy > 0

    @alive.setter
    def alive(self, value):
        if bool(self.alive) == bool(value):
            return

        if value and not self.alive:
            self._current_age = self._max_age - 1
            self._current_energy = 1
        elif not value and self.alive:
            self._current_age = self._max_age + 1

    def __bool__(self):
        return self.alive

    @classmethod
    def spawn(cls, population_size, current_age):
        obj = cls()
        obj._population_size = population_size
        obj._current_age = current_age
        return obj

    def reproduce(self):
        if not self.alive:
            print(f"Организм {self._id} мертв и не может размножаться.")
            return None

        new_energy = math.floor(self._current_energy / REPRODUCE_NEW_COEF)
        new_population_size = math.floor(self._population_size / REPRODUCE_POPULATION_COEF)

        if new_energy <= 0 or new_population_size <= 0:
            print(f"Организм {self._id} не имеет достаточно ресурсов для размножения (E: {self._current_energy}, P: {self._population_size}).")
            return None

        child = type(self)()
        child._current_age = 0
        child._current_energy = new_energy
        child._population_size = new_population_size

        self._current_energy = math.floor(self._current_energy / REPRODUCE_OLD_COEF)

        print(f"Организм {self._id} (популяция: {self._population_size}) размножился. "
              f"Появился новый организм {child._id} (популяция: {child._population_size}). "
              f"Энергия родителя уменьшена до {self._current_energy}, энергия потомка: {child._current_energy}.")
        return child

    @abstractmethod
    def __str__(self):
        pass

    @abstractmethod
    def __repr__(self):
        pass

    @abstractmethod
    def replenish_energy(self):
        pass

# 2. Concrete implementations (2 балла)

Реализуй два класса - Herbivore (травоядные) и Plant (растения). Оба должны наследоваться от Organism.

Для Herbivore:

- Инициализатор, который проставляет `max_age` равным `MAX_HERBIVORE_AGE`
- `replenish_energy(eaten_plant_energy)`, который увеличивает количество энергии на `eaten_plant_energy`.


Для Plant:

- Инициализатор, который проставляет `max_age` равным `MAX_PLANT_AGE`
- `replenish_energy` увеличивает текущий запас энергии на `PLANT_ENERGY_INC_COEF`

In [3]:
MAX_HERBIVORE_AGE = 50
MAX_PLANT_AGE = 20
PLANT_ENERGY_INC_COEF = 5

class Herbivore(Organism):
    def __init__(self):
        super().__init__()
        self._max_age = MAX_HERBIVORE_AGE

    def __str__(self):
        return (f"Herbivore ID: {self._id}, Age: {self._current_age}, "
                f"Energy: {self._current_energy}, Population: {self._population_size}, "
                f"Alive: {self.alive}")

    def __repr__(self):
        return self.__str__()

    def replenish_energy(self, eaten_plant_energy: int):
        self._current_energy += eaten_plant_energy
        print(f"Herbivore {self._id} пополнил энергию на {eaten_plant_energy}. Текущая энергия: {self._current_energy}")


class Plant(Organism):
    def __init__(self):
        super().__init__()
        self._max_age = MAX_PLANT_AGE

    def __str__(self):
        return (f"Plant ID: {self._id}, Age: {self._current_age}, "
                f"Energy: {self._current_energy}, Population: {self._population_size}, "
                f"Alive: {self.alive}")

    def __repr__(self):
        return self.__str__()

    def replenish_energy(self):
        self._current_energy += PLANT_ENERGY_INC_COEF
        print(f"Plant {self._id} пополнило энергию на {PLANT_ENERGY_INC_COEF}. Текущая энергия: {self._current_energy}")

Напиши несколько примеров работы с этими классами - протестируй таким образом, что всё ок

In [4]:
# Сбросим счетчик ID для чистоты тестового примера
ID_COUNTER = 0

print("\n--- Тестирование класса Herbivore ---")
# Создаем травоядное
herb_1 = Herbivore()
herb_1._population_size = 10
herb_1._current_energy = 15
print(f"Создан: {herb_1}")
print(f"Herbivore ID {herb_1._id} жив? {bool(herb_1)}")

# Пополняем энергию травоядного
herb_1.replenish_energy(5)
print(f"После пополнения энергии: {herb_1}")

# Травоядное размножается
herb_child = herb_1.reproduce()
if herb_child:
    print(f"Родитель после размножения: {herb_1}")
    print(f"Потомок травоядного: {herb_child}")

# Убиваем травоядное через сеттер alive
print(f"\nУбиваем Herbivore ID {herb_1._id}...")
herb_1.alive = False
print(f"Herbivore ID {herb_1._id} жив? {bool(herb_1)}")
print(f"Состояние Herbivore ID {herb_1._id} после смерти: Age={herb_1._current_age}, Energy={herb_1._current_energy}")

# Пытаемся оживить травоядное
print(f"\nПытаемся оживить Herbivore ID {herb_1._id}...")
herb_1.alive = True
print(f"Herbivore ID {herb_1._id} жив? {bool(herb_1)}")
print(f"Состояние Herbivore ID {herb_1._id} после оживления: Age={herb_1._current_age}, Energy={herb_1._current_energy}")



print("\n--- Тестирование класса Plant ---")
# Создаем растение
plant_1 = Plant()
plant_1._population_size = 20
plant_1._current_energy = 10
print(f"Создано: {plant_1}")
print(f"Plant ID {plant_1._id} живо? {bool(plant_1)}")

# Пополняем энергию растения
plant_1.replenish_energy()
print(f"После пополнения энергии: {plant_1}")

# Растение размножается
plant_child = plant_1.reproduce()
if plant_child:
    print(f"Родитель после размножения: {plant_1}")
    print(f"Потомок растения: {plant_child}")

# Проверяем условия смерти (например, возраст)
print(f"\nПроверяем смерть растения по возрасту...")
plant_2 = Plant()
plant_2._population_size = 50
plant_2._current_age = MAX_PLANT_AGE
plant_2._current_energy = 5
print(f"Создано: {plant_2}")
print(f"Plant ID {plant_2._id} живо? {bool(plant_2)}")
# Увеличиваем возраст, чтобы растение умерло
plant_2._current_age += 1
print(f"Plant ID {plant_2._id} живо после увеличения возраста? {bool(plant_2)}")

print("\n--- Тестирование воспроизведения без достаточной энергии/популяции ---")
herb_low_res = Herbivore()
herb_low_res._population_size = 1
herb_low_res._current_energy = 1
print(f"\nСоздан травоядный с низкими ресурсами: {herb_low_res}")
herb_low_res_child = herb_low_res.reproduce()
if not herb_low_res_child:
    print(f"Организм {herb_low_res._id} не смог размножиться, как и ожидалось.")


--- Тестирование класса Herbivore ---
Создан: Herbivore ID: 0, Age: 0, Energy: 15, Population: 10, Alive: True
Herbivore ID 0 жив? True
Herbivore 0 пополнил энергию на 5. Текущая энергия: 20
После пополнения энергии: Herbivore ID: 0, Age: 0, Energy: 20, Population: 10, Alive: True
Организм 0 (популяция: 10) размножился. Появился новый организм 1 (популяция: 5). Энергия родителя уменьшена до 10, энергия потомка: 10.
Родитель после размножения: Herbivore ID: 0, Age: 0, Energy: 10, Population: 10, Alive: True
Потомок травоядного: Herbivore ID: 1, Age: 0, Energy: 10, Population: 5, Alive: True

Убиваем Herbivore ID 0...
Herbivore ID 0 жив? False
Состояние Herbivore ID 0 после смерти: Age=51, Energy=10

Пытаемся оживить Herbivore ID 0...
Herbivore ID 0 жив? True
Состояние Herbivore ID 0 после оживления: Age=49, Energy=1

--- Тестирование класса Plant ---
Создано: Plant ID: 2, Age: 0, Energy: 10, Population: 20, Alive: True
Plant ID 2 живо? True
Plant 2 пополнило энергию на 5. Текущая энерг

В каком порядке должен вызываться (и вызываются у тебя):
- метод `spawn`
- `__init__` у ребёнка (`Plant/Herbivore`)
- `__init__` у родителя (`Organism`)?

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

сначала spawn, потом init у ребенка, потом init у родителя, у меня вызывается в том же порядке

# 3. Нюансы работы с классами (3 балла)


В следующих ячейках изучи вопросы копирования классов как объектов (НЕ экземпляров! Именно самих классов) и последующего их сравнения. Также изучи, можно ли поменять `__bases__` и/или `__mro__` после создания класса как объекта. Если что-то сходу не получается - погугли :)

In [ ]:
# TODO

Какие выводы ты можешь сделать? Какие есть нюансы:

1. Копирования классов?

2. Сравнения классов?

3. Изменения `__bases__` и `__mro__` после создания класса?

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

Теперь изучи следующие два примера и проанализируй.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class A:
    pass

class B:
    pass

B.__bases__ = (A, )


In [ ]:
class A:
    pass

class B:
    pass

class C(A):
    pass

C.__bases__ = (B, )


Выводы:

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

Теперь попробуй посоздавать экземпляры классов и поменять у них классы динамически. Всегда ли это получается? Почему?

In [ ]:
class A:
    pass # напиши свою базовую реализацию (пару методов), чтобы проверить работоспособность

class B:
    pass

a = A()
a.__class__ = B
# TODO
a.__class__ = int

Выводы:

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

# 4. Mixin'ы. (5 баллов)


Наша цель - в Runtime добавлять к классам новые характеристики, логика которых будет описана в Mixin'ах.

Подумай, как надо поменять реализацию классов `Herbivore` и `Plant`, чтобы легко вызывались методы из Mixin'ов, но при этом если их нет, тоже ничего не ломалось.

Как это сделать?

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

Скопируй `Herbivore` и `Plant` и добавь изменения в них, чтобы работало корректно в будущем

In [ ]:
# TODO

Напиши следующие Mixin'ы:

- `ShortLifespan` - уменьшает максимальный возраст организма в `SHORT_LIFESPAN_COEF` раз, но увеличивает прибавку энергии в `SHORT_LIFESPAN_ENERGY_COEF` раз

- `LongLifespan` - обратное предыдущему. Добавь соответствующие константы.

- `HighFertilityMixin` - увеличивает у возвращаемого "ребёнка" в методе `reproduce` `population_size` в `HIGH_FERTILITY_COEF` раз, но повышает затраты энергии в `reproduce` в `HIGH_FERTILITY_ENERGY_COEF` раз.

- `Toxic` (только для растений) - при поедании такого растения количество животных из данного набора уменьшается в `TOXIC_COEF` раз.

- `IdentifiesToxicPlants` (только для травоядных) - при выборе растения в фазе поедания, если растение токсично, выбирают не есть его.

- `ResilientToToxicPlants` (только для травоядных) - при выборе токсичного растения смертность уменьшена в `RESILIENT_COEF` раз.

In [ ]:
# TODO

Перекопируй код для абстрактного класса из п. 1 данной работы и добавь к нему метод `mutate`: с вероятностью `MUTATION_POSSIBILITY` процентов он выбирает какой-то из mixin'ов, который у него ещё не стоит в родителях, и добавляет его в родителей; иначе - он выбирает одного из родителей и удаляет его из кортежа родителей (`__bases__`).


Подумай, как это реализовать так, чтобы сам метод был общий и для `Plant`, и для `Herbivore`, но при этом выбор возможных Mixin'ов был разный. При надобности их тоже скопируй ниже и поправь в них код.


Также важно! в методе `mutate` должно быть логгирование (`print` с описанием произошедшего), чтобы было понятно, кто мутировал (и во что).

In [ ]:
# TODO

Напиши пару примеров работы с `mutate`, чтобы протестировать, что всё сделано хорошо.

In [ ]:
# TODO

# 5. Собираем симуляцию вместе (5 баллов)


Напиши класс `Game`, который будет хранить список текущих животных и растений. Заполни его на своё усмотрение в начале игры (метод `__init__`), НО в начале не должно быть ни одного животного/растения с мутациями (mixin'ами).

У него должны быть:

- метод `__init__` (см. выше)

- метод `correct_age_energy`, который итерируется по всем живым организмам и делает с каждым следующие шаги:

1. Вычитает константу потребления энергии (они различны для животных и растений)
2. Прибавляет age на 1.
3. Если животное или растение стало слишком старым ИЛИ энергия стала <= 0, то оно умирает (и это логируется в поток вывода).


- метод `cycle_through_actions`, который итерируется по всем живым организмам и с каждым равновероятно делает одно из трёх действий: `replenish_energy` (для животных оно выбирает растение и уменьшает его популяцию в `PLANT_EATEN_COEF` и энергию в `PLANT_ENERGY_EATEN_COEF` раз, после чего повышает собственную энергию через `replenish_energy`), `reproduce`, `mutate`. Каждый из этих трёх действий должно корректно логироваться (чтобы мы понимали, что происходит в симуляции).


- метод `play_game(n)` - который запускает игру и делает последовательно `n` циклов жизни (запусков `correct_age_energy` и `cycle_through_actions`).

In [ ]:
# TODO

Собери все возможные коэффициенты и установи их на своё усмотрение в ячейке ниже.

Запусти игру. Какие результаты получились после запуска?)

In [ ]:
# TODO

Результаты

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>